In [5]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 57.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
# 파일 형식 확인
import chardet

with open('/content/drive/MyDrive/train_game.json', 'rb') as f:
    result = chardet.detect(f.read())

print(result['encoding'])

utf-8


In [6]:
import json

def open_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

In [8]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time

# 측정 시작
start = time.time()

# 리뷰 파일 읽어오기
print('1) 말뭉치 데이터 읽기 시작')
train = open_json("/content/drive/MyDrive/train_game.json")
print(len(train)) # 리뷰 데이터 전체 개수
print('1) 말뭉치 데이터 읽기 완료: ', time.time() - start)

1) 말뭉치 데이터 읽기 시작
863
1) 말뭉치 데이터 읽기 완료:  0.11957478523254395


In [10]:
# 문장단위로 명사만 추출해 학습 입력 데이터로 만듬
print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran()
docs = [komoran.nouns(sentence["sentence"]) for sentence in train]
print('2) 형태소에서 명사만 추출 완료: ', time.time() - start)

2) 형태소에서 명사만 추출 시작
2) 형태소에서 명사만 추출 완료:  41.92937183380127


In [11]:
# word2vec 모델 학습
print('3) word2vec 모델 학습 시작')
model = Word2Vec(sentences=docs, vector_size=200, window=4, min_count=2, sg=1) # skip-gram
print('3) word2vec 모델 학습 완료: ', time.time() - start)

3) word2vec 모델 학습 시작
3) word2vec 모델 학습 완료:  52.89315843582153


In [12]:
# 모델 저장
print('4) 학습된 모델 저장 시작')
model.save('nvmc.model')
print('4) 학습된 모델 저장 완료: ', time.time() - start)

# 학습된 말뭉치 개수, 코퍼스 내 전체 단어 개수
print("corpus_count : ", model.corpus_count)
print("corpus_total_words : ", model.corpus_total_words)

4) 학습된 모델 저장 시작
4) 학습된 모델 저장 완료:  62.355417013168335
corpus_count :  863
corpus_total_words :  7576


In [15]:
model = Word2Vec.load('nvmc.model')
print(model.wv.most_similar("퀘스트", topn = 5))
print(model.wv.most_similar("나이트", topn = 5))

[('거', 0.7553879022598267), ('때', 0.72751384973526), ('스키', 0.7222560048103333), ('것', 0.7182260155677795), ('드', 0.7125344276428223)]
[('주문', 0.46636277437210083), ('쯤', 0.41791871190071106), ('레', 0.4165864884853363), ('드', 0.4121136963367462), ('거', 0.40452924370765686)]
